## Introduction

This is a tutorial for an n-body simulator using Scipy's ordinary differential equation integrator and gradient calculator functions complete with animation tools in plotly. This simulator is 

In [1]:
# Import necessary packages
import numpy as np
import scipy
from scipy.optimize import approx_fprime
from scipy import integrate
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Define some constants for examples
au2m       = 1.49598e11 # AU in meters
days2sec    = 86400. # Days in seconds
years2sec    = 31557600. # Years in seconds
mSun_kg     = 1.98892e30 # Mass of Sun in Kg
mEarth_kg   = 5.9742e24 # Mass of Earth in Kg
mJupiter_kg = 1.8987e27 # Mass of Jupiter in Kg
rSun_m = 6.9634e8 # Radius of Sun in meters
rJupiter_m = 6.9911e7 # Radius of Jupiter in meters
rEarth_m = 6.371e6 # Radius of Earth in meters
pMass = 1.6723e-27 # Mass of Proton in Kg
eMass = 9.1094e-31 # Mass of Electron in Kg
eCharge = 1.6e-19 # Elementary Charge in Coulombs
hDist = 5.29e-11 # Distance of Electron to nucleus in Hydrogen in meters
eVel = 2.18e6 # Velocity of electron around hydrogen nucleus in m/s
Earth_vel = 29784.8 # Velocity of Earth around Sun in m/s
G = 6.67e-11 # Gravitational Constant
K = 9.0e+9 # Coulomb Constant

## System and Particle Classes

Below, we define particle and system classes to store the state and properties of the system we aim to simulate. These properties include:
1. List of particles with mass, position, velocity, and charge
2. System force type (Gravitational, Coulomb, Spring)
3. Coupling strength between particles
4. Equilibrium of spring force(s)

In [ ]:
# Particle Class
class particle:
    def __init__(self, mass, init_pos, init_vel, charge):
        if len(init_vel) != len(init_pos):
            raise Exception("ERROR: Velocity and/or Position are not the same dimensions!")
        if len(init_vel) != 3 or len(init_pos) != 3:
            raise Exception("ERROR: Velocity and/or Position are not 3D!")
        if type(mass) != int and type(mass) != float:
            raise Exception("ERROR: Mass is not a number!")
        if type(charge) != int and type(charge) != float:
            raise Exception("ERROR: Mass is not a number!")
        if mass < 0:
            raise Exception("ERROR: Mass is negative!")

            
        self.mass = mass
        self.charge = charge
        self.pos = np.array(init_pos)
        self.vel = np.array(init_vel)
        
class system:
    def __init__(self, p_list, coupling, force_type, eq):
        if len(p_list) == 0:
            raise Exception("ERROR: No particles in system")
        if np.shape(coupling)[0] != np.shape(coupling)[1]:
            raise Exception("ERROR: Coupling matrix not square!")
        if np.shape(eq)[0] != np.shape(eq)[1]:
            raise Exception("ERROR: Spring equilibrium matrix not square!")
            
        self.p_list = np.array(p_list) # Particle list
        self.type = force_type # 0 = Gravity, 1 = Charge, 2 = Spring 
        self.coupling = coupling # 2D array of force coupling between particles
        self.eq = eq # 2D array of spring equilibrium distance
        self.size = len(p_list)
    def size(self):
        return self.size

Below we define functions for:
1. pot_func: the potentials of the three forces we aim to simulate
2. force_calculator: a calculator of the forces between all particles of a system-state for said forces (using Scipy's gradient function Approx_fprime)
3. int_sys: a function that takes in a system and returns the velocity and acceleration vectors of the particles, fed to Scipy's ODE integrator. Note: The syntax of "x" in the function contains both position and velocity of the particles. This is an explicit argument of the function alongisde the system because ode_int requires the integrated ODE function to take in position and velocity and return the time-derivates of them.

In [ ]:
def pot_func(dist, p1, p2, force_type, coupling, equi):
    # Potential function for gravitational potential  (0), electrostatic forces (1), and spring (2)
    # Force type == 0 for gravitational, 1 for Coulombic, 2 for spring potential
    # Coupling is a 2D array for the strength of all particle interactions;
    # If 0, then particles do not interact. Any other number scales interaction by that value
    if force_type == 0:
        potential = coupling * (G * p1.mass * p2.mass) / (dist)
    elif force_type == 1:
        potential = coupling * -1*(K * p1.charge * p2.charge) / (dist)
    elif force_type == 2:
        potential = -1*((coupling / 2) * (dist - equi) ** 2)
    else:
        raise Exception("Force type invalid!")
    return potential

        
def force_calculator(psystem): # Calculate forces between all particles, put into an array
    forces = np.zeros((len(psystem.p_list), 3))
    forces = np.array(forces)

    # Loop over all possible pairs of particles without double counting
    for i in range(0,len(psystem.p_list)):
        for j in range(i+1,len(psystem.p_list)): 
            m1 = psystem.p_list[i].mass # Masses
            m2 = psystem.p_list[j].mass
            r = psystem.p_list[i].pos - psystem.p_list[j].pos # Vector
            dist = np.linalg.norm(r) # Distance Magnitude
            # Use Scipy's Approx_fprime to calculate gradient i.e. force from potential function
            force = approx_fprime(dist, pot_func, dist, psystem.p_list[i], psystem.p_list[j], psystem.type, psystem.coupling[i][j], psystem.eq[i][j]) # Force vector between two particles
            # Sum net force, keeping in mind Newton's 3rd Law
            forces[i] += force * r / dist
            forces[j] -= force * r / dist
    return forces   

# System integrator function to be fed into Scipy's Odeint
# Take in the positions and velocities, the system, and the time array
# and returns velocities and accelerations
def int_sys(x, t, psystem): 
    n = int(len(x)/2) # Split position and velocity array
    p = np.array(x[:n])
    v = np.array(x[n:])
    for i in range(0,psystem.size):
        # Update position and velocity of the particles
        psystem.p_list[i].pos = p[3*i:3*i+3]
        psystem.p_list[i].vel = v[3*i:3*i+3]
    vel = np.array(np.zeros((len(psystem.p_list), DIMENSIONS)))
    part_count = 0

    # 
    i = 0
    while i < len(v):
        for j in range(0, DIMENSIONS):
            vel[part_count][j] = v[i]
            i += 1
        part_count += 1
    
    accels = np.zeros((len(psystem.p_list), DIMENSIONS))
    accels = np.array(accels)
    # Get net force on all particles from current system state
    forces = np.array(force_calculator(psystem))
    # Convert forces into accelerations
    for i in range(0, psystem.size):
        accels[i] = forces[i]/psystem.p_list[i].mass
    # print(accels)
    a = np.ravel(accels)
    v = np.ravel(vel)
    # print(a, v)
    return np.concatenate((v, a))

## Visualization / Animation

Below is the function to create a user-controlled animation of the trajectories in the system over a given time. 

In [ ]:
def nBody_animation(num_particles, traj, t):
    # Given the array-like input variable t, initialize variables that store
    # the number of time steps and the length of each step.
    n_timesteps = len(t)
    total_time = int(max(t))
    dt = round((total_time/n_timesteps), 2)
    # Initialize array of particle position data - random for now
    positions = np.random.randn(n_timesteps, num_particles, 3)
    # Iterate through array traj to replace values in positions array with those in traj
    for n in range(0,n_timesteps):
        for i in range(0,num_particles):
            for c in range(0,3):
                positions[n,i,c] = traj[n,3*i+c]
    # Create x, y, and z ranges based on the min, max values of x, y, and z
    # That appear in the positions array
    x_range = np.array([np.min(positions[:,:,0]), np.max(positions[:,:,0])])
    y_range = np.array([np.min(positions[:,:,1]), np.max(positions[:,:,1])])
    z_range = np.array([np.min(positions[:,:,2]), np.max(positions[:,:,2])])
    # Initialize the figure
    layout = go.Layout(width=600, height=600, title="computed N-body")
    fig = go.Figure(layout=layout)
    # Assign unique random colors to each particle
    # And add the initial plot (time step zero)
    particle_colors = {}
    for iparticle in range(num_particles):
        rgb = np.random.randint(0, 255, 3)
        color = f'rgb({rgb[0]},{rgb[1]},{rgb[2]})'
        particle_colors[iparticle] = color
        # Add initial scatter plot (marker)
        fig.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers',
            marker=dict(size=5, color=color),
            name=f'Particle {iparticle}'
        ))
        # Add trajectory line
        fig.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode='lines',
            line=dict(color=color, width=2),
            name=f'Trajectory {iparticle}'
        ))
    # Assign length of trajectory tail
    tail_length = 1000
    # Create frames for each time step
    frames = []
    for k in range(n_timesteps):
        frame_data = []
        for iparticle in range(num_particles):
            # Current positions of particles (scatter plot)
            frame_data.append(go.Scatter3d(
                x=np.array(positions[k, iparticle, 0]),
                y=np.array(positions[k, iparticle, 1]),
                z=np.array(positions[k, iparticle, 2]),
                mode='markers',
                marker=dict(size=5, color=particle_colors[iparticle]),
                name=f'Particle {iparticle + 1}'
            ))
            # Particle trajectory tails (lines)
            start_index = max(0, k - tail_length + 1)  # Ensure we don’t try to make a trajectory start before 0
            frame_data.append(go.Scatter3d(
                x=positions[start_index:k+1, iparticle, 0],
                y=positions[start_index:k+1, iparticle, 1],
                z=positions[start_index:k+1, iparticle, 2],
                mode='lines',
                line=dict(color=particle_colors[iparticle], width=2),
                name=f'Trajectory {iparticle + 1}'
            ))
        frames.append(go.Frame(data=frame_data, name=f'frame{k}'))
    fig.frames = frames
    # Add animation controls
    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            buttons=[
                dict(label="Play",
                     method="animate",
                     args=[None, {"frame": {"duration": 50, "redraw": True},
                                  "fromcurrent": True, "transition": {"duration": 0}}]),
                dict(label="Pause",
                     method="animate",
                     args=[[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])
            ],
            direction="left",
            pad={"r": 10, "t": 87},
            showactive=False,
            x=0.1, xanchor="right", y=0, yanchor="top"
        )],
        sliders=[dict(
            active=0,
            yanchor="top", xanchor="left",
            currentvalue={"prefix": "Time: ", "suffix": " s"},
            pad={"b": 10, "t": 50}, len=0.9, x=0.1, y=0,
            steps=[{"method": "animate",
                    "args": [[f'frame{k}'],
                             {"frame": {"duration": 10, "redraw": True},
                              "mode": "immediate",
                              "transition": {"duration": 0}}], "label": f"{k * dt:.2f}"}
                   for k in range(n_timesteps)]
        )]
    )
    # Update initial layout to include the custom axis ranges set above
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='x (m)', range=[x_range[0], x_range[1]], autorange=False),
            yaxis=dict(title='y (m)', range=[y_range[0], y_range[1]], autorange=False),
            zaxis=dict(title='z (m)', range=[z_range[0], z_range[1]], autorange=False),
            aspectratio=dict(x=1, y=1, z=1)
        )
    )
    return fig

## Spring Example

### A 3 dimensional simulation of n-body systems with springs. Feel free to change the initial conditions of the system below!

In [ ]:
# Set constants and initial state of the system
DIMENSIONS = 3 # Dimensions to model the system
num_particles = 2 # Number of particles
# Force coupling value between all particles (0 for no interactions, f1 for full interaction, 0-1 for force adjusted by a fraction)
coupling = [[0, 10],
            [10, 0]]

eq = [[0, 1],
     [1, 0]]
# Masses of the particles
masses = [1, 1]

# Initial positions of the particles
positions = [[1, 4, 5], [1, 7, 8]]
# Initial velocities of the particles
velocities = [[0, 0, 0], [0, 0, 0]]
# Charges of particles
charges = [0, 0]

# Charges of the particles
p = []
for i in range(0, num_particles):
    p.append(particle(masses[i], positions[i], velocities[i], charges[i]))
# Create particle objects from the particle class and create a system of particles
psystem = system(p, coupling, 2, eq)

# Sanity Check for class initialization
# for i in range(num_particles):
#     print("Position: " + str(psystem.p_list[i].pos))
#     print("Velocity: " + str(psystem.p_list[i].vel))
#     print("Mass: " + str(psystem.p_list[i].mass))
#     print("Charge: " + str(psystem.p_list[i].charge))

# forces = force_calculator(psystem)
# print(forces) #works now!

# Create array of positions and velocities
pos_vel = []
for i in range(0, psystem.size):
    for j in range(0,3):
        pos_vel.append(psystem.p_list[i].pos[j])
        
for i in range(0, psystem.size):
    for j in range(0,3):
        pos_vel.append(psystem.p_list[i].vel[j])
        
t = np.linspace(0,10,1000)

# Integrate system over time t
traj = scipy.integrate.odeint(int_sys, pos_vel, t, args=(psystem,))

In [ ]:
# fig_nbody_animation = nBody_animation(num_particles, traj, t)
# fig_nbody_animation.show()

## Gravity Example

### A 3 dimensional simulation of n-body systems with Gravity. Feel free to change the initial conditions of the system below!

In [ ]:
# Set constants and initial state of the system
DIMENSIONS = 3 # Dimensions to model the system
num_particles = 2 # Number of particles
# force coupling value between all particles (0 for no interactions, 1 for full interaction, 0-1 for force adjusted by a fraction)
coupling = np.ones([num_particles, num_particles]) 

eq = np.ones([num_particles, num_particles])
# Masses of the particles
masses = [1*mSun_kg, 1*mSun_kg]
# Initial positions of the particles
positions = [[-0.3*au2m, -0.3*au2m, 0.1*au2m], [0.207*au2m, 0.207*au2m, 0.3*au2m]]
# Initial velocities of the particles
velocities = [[0, 0, 0], [0.707*Earth_vel, -0.707*Earth_vel, 0]]
# Charges of particles
charges = [0, 0]

# Charges of the particles
p = []
for i in range(0, num_particles):
    p.append(particle(masses[i], positions[i], velocities[i], charges[i]))
# Create particle objects from the particle class and create a system of particles
psystem = system(p, coupling, 0, eq)

# Sanity Check for class initialization
# for i in range(num_particles):
#     print("Position: " + str(psystem.p_list[i].pos))
#     print("Velocity: " + str(psystem.p_list[i].vel))
#     print("Mass: " + str(psystem.p_list[i].mass))
#     print("Charge: " + str(psystem.p_list[i].charge))

# forces = force_calculator(psystem)
# print(forces) #works now!

# Create array of positions and velocities
pos_vel = []
for i in range(0, psystem.size):
    for j in range(0,3):
        pos_vel.append(psystem.p_list[i].pos[j])
        
for i in range(0, psystem.size):
    for j in range(0,3):
        pos_vel.append(psystem.p_list[i].vel[j])
        
t = np.linspace(0,1*years2sec,1000)

# Integrate system over time t
traj = scipy.integrate.odeint(int_sys, pos_vel, t, args=(psystem,))

In [ ]:
# fig_nbody_animation = nBody_animation(num_particles, traj, t)
# fig_nbody_animation.show()

## Coulomb Example

### A 3 dimensional simulation of n-body systems with Coulomb forces. Feel free to change the initial conditions of the system below!

In [ ]:
# Set constants and initial state of the system
DIMENSIONS = 3 # Dimensions to model the system
num_particles = 2 # Number of particles
# Force coupling value between all particles (0 for no interactions, 1 for full interaction, 0-1 for force adjusted by a fraction)
coupling = np.ones([num_particles, num_particles]) 


eq = np.zeros([num_particles, num_particles])
# Masses of the particles
masses = [pMass, eMass]
# Initial positions of the particles
positions = [[0, 0, 0], [0, hDist, 0]]
# Initial velocities of the particles
velocities = [[0, 0, 0], [eVel, 0, 0]]
# Charges of particles
charges = [eCharge, -eCharge]

# Charges of the particles
p = []
for i in range(0, num_particles):
    p.append(particle(masses[i], positions[i], velocities[i], charges[i]))
# Create particle objects from the particle class and create a system of particles
psystem = system(p, coupling, 1, eq)

# Sanity Check for class initialization
# for i in range(num_particles):
#     print("Position: " + str(psystem.p_list[i].pos))
#     print("Velocity: " + str(psystem.p_list[i].vel))
#     print("Mass: " + str(psystem.p_list[i].mass))
#     print("Charge: " + str(psystem.p_list[i].charge))

# forces = force_calculator(psystem)
# print(forces) #works now!

# Create array of positions and velocities
pos_vel = []
for i in range(0, psystem.size):
    for j in range(0,3):
        pos_vel.append(psystem.p_list[i].pos[j])
        
for i in range(0, psystem.size):
    for j in range(0,3):
        pos_vel.append(psystem.p_list[i].vel[j])
        
t = np.linspace(0,3e-16,1000)

# Integrate system over time t
traj = scipy.integrate.odeint(int_sys, pos_vel, t, args=(psystem,))

In [ ]:
# fig_nbody_animation = nBody_animation(num_particles, traj, t)
# fig_nbody_animation.show()